# Step 1b: Generate forcing for any region from CMIP prediction ensemble for HBV model

In this notebook an ensemble of forcing (time dependent input data) for the HBV model is created from a ensemble of CMIP6 climate model runs. If a (calibrated) hydrological model is available this ensemble can be used to analyse how the (statistics of) river discharge will change with climate change.

We choose one climate model (MPI-ESM1-2-LR) and one scenario (ssp585) for this example notebook, but a list of models and / or scenarios can be provided. The meta-data on these must be available through the output of notebook 0: a collection of json files in the configFiles directory. 

In this example notebook we generate forcing for the HBV model for the entire Rhine basin. From a hydrological viewpoint this is a bad decision: the Rhine basin is too large to be modeled by a single HBV model: all spatial values are averaged over the entire basin and this is obviously too much of a simplification. We choose this combination in this example for two reasons:

- The point of this notebook is to demonstrate the workflow and not the hydrological outcome. Therefore an unrealistic example was chosen. The folder ```examples``` contain model notebooks with different models.
- While many hydrological models are computationally heavy, the HBV models is relativly light, which allows anyone with relative simple infrastructure (compute power / memory) to run this notebook. 

For now you have to supply your own shapefile of the basin. We have already uploaded a shapefile for the Rhine in the configFiles directory, to be able to test this notebook ourselves.

We have seperated generating the forcing and running the model into two notebooks because generating the forcing is quite heavy on the hardware and we want to minimize the number of times you have to do that. After generating your forcing in this notebook, it will be saved so we can load it later without having to re-run it.

In [1]:
# eWaterCycle dependencies. 
# note that we do not need to import the model when only generating forcingimport ewatercycle.forcing
import ewatercycle.forcing
import ewatercycle.analysis

# other dependencies
from pathlib import Path
from cartopy.io import shapereader
import pandas as pd
import numpy as np
from rich import print
import json


/opt/conda/envs/ewatercycle2/lib/python3.10/site-packages/esmvalcore/experimental/_warnings.py:13: UserWarning: 
  Thank you for trying out the new ESMValCore API.
  Note that this API is experimental and may be subject to change.
  More info: https://github.com/ESMValGroup/ESMValCore/issues/498


In [2]:
#info on the region and the time periods of interest

#Station ID for the Rhine at Lobith
grdc_station_id = "6335020"

# future climate predication information
prediction_start_time = "2024-01-01T00:00:00Z"
prediction_end_time = "2049-12-31T00:00:00Z"

# selection of climate model(s) and experiments. Confusingly, climate models are called
# datasets in ESGF/CMIP/ESMValTool jargon.
datasets = ['MPI-ESM1-2-LR']
exps = ['ssp585']

# location where the json input files are located (from notebook 0)
config_file_location = Path.cwd() / "configFiles"

#location to save forcing results
path = Path.cwd()
forcing_path = path / "Forcing"
saveLocationDirName = "GRDCStation" + grdc_station_id + "CMIPPrediction"

In [3]:
#TODO: add shapeFiles to the meta-data of get_grdc_data (or even better: GRDC API call?)
path = Path("/home/rhut/metaFiles")
shapeFileName = "Rhine.shp"
#shapeFileName = "grdc_basins_smoothed_md_no_"+str(grdc_station_id)+".shp"
catchmentShapeFile = path / shapeFileName



In [4]:
# here we generate the actual forcings. Files from CMIP models are stored on ESGF nodes. These 
# nodes are not always online. Therefore, for each ensemblemember we put the line to 
# generate the forcing data in a ```try``` / ```except``` statement. If the creation of forcing 
# failes, the error is captured and written in an error.log file.

for dataset in datasets:
    for exp in exps:
        
        json_file = "datasets_" + dataset + "_" + exp + ".json"
        with (open(config_file_location / json_file, 'r')) as the_file:
            ensembles = json.load(the_file)

        succesfull_ensembles = {}

        for ensemble in ensembles:
            saveLocationCMIPPrediction = forcing_path / saveLocationDirName / dataset / ensemble

            try:
                CMIP6_forcing_prediction = ewatercycle.forcing.sources["LumpedMakkinkForcing"].generate(
                    dataset=ensembles[ensemble],
                    start_time=prediction_start_time,
                    end_time=prediction_end_time,
                    shape=catchmentShapeFile.absolute(),
                    directory = saveLocationCMIPPrediction
                )
                print(dataset + " ensemble " + ensemble + " done")

                succesfull_ensembles[ensemble] = ensembles[ensemble]
                
            except Exception as err: 
                print(dataset + " ensemble " + ensemble + " failed")

                #create textfile 
                with (open(saveLocationCMIPPrediction / "error.log", 'w')) as the_file:
                     the_file.write(str(err))


        with (open(config_file_location / json_file, 'w')) as the_file:
             json.dump(succesfull_ensembles, the_file)
            
        




MPI-ESM1-2-LR ensemble r34i1p1f1 done

MPI-ESM1-2-LR ensemble r10i1p1f1 done

MPI-ESM1-2-LR ensemble r19i1p1f1 done

MPI-ESM1-2-LR ensemble r16i1p1f1 done

MPI-ESM1-2-LR ensemble r26i1p1f1 done

...

MPI-ESM1-2-LR ensemble r29i1p1f1 done

MPI-ESM1-2-LR ensemble r24i1p1f1 done

MPI-ESM1-2-LR ensemble r40i1p1f1 done

MPI-ESM1-2-LR ensemble r32i1p1f1 done

MPI-ESM1-2-LR ensemble r37i1p1f1 done